In [2]:
import os

data_path = "data/capture_drone_steady"

if not os.path.exists(data_path):
    print("file does not exist")
else:
    print("file exists. path: {}".format(data_path))

file path: data/capture_drone_steady


In [3]:

# declaring the variables

# Sensor Configuration
## Profile config
start_freq = 77  # GHz 
slope = 79  # MHz/us
idle_time = 5  # us
adc_start_time = 6  # us
adc_samples = 256  # Number of samples per chirp
sample_freq = 8000  # ksps (8 MSPS)
ramp_end_time = 40  # us
rx_gain = 48  # dB

## Frame config
### Chirp config
start_chirp_tx = 0
end_chirp_tx = 11
nchirp_loops = 64
nframes_master = 0  # Number of Frames for Master (0 means continuous streaming mode)
nframes_slave = 0  # Number of Frames for Slaves (0 means continuous streaming mode)
Inter_Frame_Interval = 100  # ms
trigger_delay = 0  # us
nDummy_chirp = 0
trig_list = {1, 2, 2, 2}  # 1: Software trigger, 2: Hardware trigger 

# Derived config
numAdcSamples = adc_samples
numRxPerDevice = 4
numDevice = numRxPerDevice
numChirpsPerFrame = end_chirp_tx - start_chirp_tx + 1
numChirpLoops = nchirp_loops
numChirpsPerFramePerDevice = numChirpsPerFrame * numChirpLoops
totalSamplePerFramePerDevice = numChirpsPerFramePerDevice * numAdcSamples * numRxPerDevice  

print(f"{numAdcSamples}, {numRxPerDevice}, {numChirpsPerFramePerDevice}, {totalSamplePerFramePerDevice}") # checking the values


256, 4, 768, 786432


## Parsing the ADC data

parsing the binary adc data

In [ ]:
import numpy as np

# parsing the binary data
def parse_adc_data(file_path, num_frames, num_chirps_per_frame, num_rx, num_samples):
    adc_data = np.fromfile(file_path, dtype=np.int16)
    
    # Each sample consists of I and Q components
    adc_data = adc_data.reshape(-1, 2)
    adc_data_complex = adc_data[:, 0] + 1j * adc_data[:, 1]
    
    # Calculate total number of chirps
    total_chirps = num_frames * num_chirps_per_frame
    
    # Reshape the data into [Frames][Chirps][RX Antennas][Samples]
    adc_data_complex = adc_data_complex.reshape(num_frames, num_chirps_per_frame, num_rx, num_samples)
    return adc_data_complex



def calculate_num_frames(file_path, num_chirps_per_frame, num_rx, num_adc_samples):
    file_size_bytes = os.path.getsize(file_path)
    # Each sample is 2 bytes for I and 2 bytes for Q = 4 bytes per complex sample
    bytes_per_sample = 4
    samples_per_frame = num_chirps_per_frame * num_rx * num_adc_samples
    bytes_per_frame = samples_per_frame * bytes_per_sample
    total_frames = file_size_bytes // bytes_per_frame
    return total_frames


# parsing data for each device

devices = ['master', 'slave1', 'slave2', 'slave3']
device_data = {}

for device in devices:
    data_files = [f for f in os.listdir(data_path) if f.startswith(device) and 'data.bin' in f]
    data_files.sort()  # Ensure files are in order
    adc_data_list = []
    for file_name in data_files:
        full_file_path = os.path.join(data_path, file_name)
        num_frames = calculate_num_frames(file_path=full_file_path, num_chirps_per_frame=numChirpsPerFrame, num_rx=numRxPerDevice, num_samples=numAdcSamples)
        adc_data = parse_adc_data(full_file_path, num_frames, num_chirps_per_frame=numChirpsPerFrame, num_rx=numRxPerDevice, num_samples=numAdcSamples)
        adc_data_list.append(adc_data)
    # Concatenate data from multiple files
    device_data[device] = np.concatenate(adc_data_list, axis=0)  # Concatenate along frames



In [ ]:
chirp_tx_mapping = {
    0: {'device': 'slave3', 'tx': 'TX2'},
    1: {'device': 'slave3', 'tx': 'TX1'},
    2: {'device': 'slave3', 'tx': 'TX0'},
    3: {'device': 'slave2', 'tx': 'TX2'},
    4: {'device': 'slave2', 'tx': 'TX1'},
    5: {'device': 'slave2', 'tx': 'TX0'},
    6: {'device': 'slave1', 'tx': 'TX2'},
    7: {'device': 'slave1', 'tx': 'TX1'},
    8: {'device': 'slave1', 'tx': 'TX0'},
    9: {'device': 'master', 'tx': 'TX2'},
    10: {'device': 'master', 'tx': 'TX1'},
    11: {'device': 'master', 'tx': 'TX0'},
}
